In [0]:
#Using dbutils.fs.ls command to check the file that been uploaded

dbutils.fs.ls("/FileStore/tables/")

Out[1]: [FileInfo(path='dbfs:/FileStore/tables/account-models/', name='account-models/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts/', name='accounts/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592, modificationTime=1706721501000),
 FileInfo(path='dbfs:/FileStore/tables/accounts2/', name='accounts2/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations/', name='activations/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/activations.zip', name='activations.zip', size=8411369, modificationTime=1706718610000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023/', name='clinicaltrial_2023/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023-1.zip', name='clinicaltrial_2023-1.zip', size=52785750, modificationTime=1712308832000),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023.zip', n

Defining Reusable code to unzip all files

In [0]:
fileroot = "clinicaltrial_2023"

dbutils.fs.cp("FileStore/tables/" + fileroot + ".zip", "file:/tmp/")
#import os library

import os
os.environ['fileroot'] = fileroot


In [0]:
%sh

unzip -d /tmp /tmp/$fileroot.zip

Archive:  /tmp/clinicaltrial_2023.zip
  inflating: /tmp/clinicaltrial_2023.csv  


In [0]:
# List the contents of the /tmp/ directory
files_in_tmp = dbutils.fs.ls("file:/tmp/")

# Print the list of files and directories in /tmp/
for file_info in files_in_tmp:
    print(file_info.path)

file:/tmp/hsperfdata_root/
file:/tmp/RtmpY3DE5Q/
file:/tmp/.XIM-unix/
file:/tmp/systemd-private-c7cfb6585b394c1cad096cfd2865a8a9-ntp.service-udjukf/
file:/tmp/systemd-private-c7cfb6585b394c1cad096cfd2865a8a9-apache2.service-iwlFrh/
file:/tmp/.ICE-unix/
file:/tmp/clinicaltrial_2023.zip
file:/tmp/systemd-private-c7cfb6585b394c1cad096cfd2865a8a9-systemd-resolved.service-auIGUg/
file:/tmp/custom-spark.conf
file:/tmp/chauffeur-daemon.pid
file:/tmp/.Test-unix/
file:/tmp/driver-daemon.pid
file:/tmp/.X11-unix/
file:/tmp/chauffeur-daemon-params
file:/tmp/python_lsp_logs/
file:/tmp/tmp.OnFqowtW1c
file:/tmp/chauffeur-env.sh
file:/tmp/systemd-private-c7cfb6585b394c1cad096cfd2865a8a9-systemd-logind.service-tGinFg/
file:/tmp/.font-unix/
file:/tmp/driver-daemon-params
file:/tmp/clinicaltrial_2023.csv
file:/tmp/Rserv/
file:/tmp/driver-env.sh


In [0]:
#Make directory

dbutils.fs.mkdirs("FileStore/tables/" + fileroot)

#Move files

dbutils.fs.mv("file:/tmp/" + fileroot + ".csv" , "/FileStore/tables/" + fileroot , True)

# remove the .zip file

dbutils.fs.rm("/FileStore/tables/" + fileroot + ".zip")

dbutils.fs.ls("/FileStore/tables/" + fileroot)

Out[5]: [FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2023/clinicaltrial_2023.csv', name='clinicaltrial_2023.csv', size=202413369, modificationTime=1713124857000)]

In [0]:
file = "/FileStore/tables/clinicaltrial_2023/clinicaltrial_2023.csv"

#dbutils.fs.head(file)

In [0]:
# Create an rdd

rdd1 = sc.textFile(file)

rdd1.take(3)

Out[18]: ['NCT Number \\t Study Title \\t Acronym \\t Study Status \\t Conditions \\t Interventions \\t Sponsor \\t Collaborators \\t Enrollment \\t Funder Type \\t Study Type \\t Study Design,Start Date,Completion Date',
 '"NCT03630471 \\t Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India \\t PRIDE \\t COMPLETED \\t Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) \\t BEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care \\t Sangath \\t Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine \\t 250 \\t OTHER \\t INTERVENTIONAL \\t Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019',
 '"NCT05992571 \\t Oral Ketone Monoester Supplementation and Resting-state Brain Connectivity \\t RECRUITING \\t Cerebrovascular Function|Co

In [0]:
colRDD = rdd1 \
        .flatMap(lambda x: x.split("\n")) \
        .filter(lambda x: x.count("\\t") >= 11) \
        .map(lambda x: x.split("\\t")) \

colRDD.take(4)
df = colRDD.toDF()

In [0]:
colRDD.take(4)

Out[28]: [['NCT Number ',
  ' Study Title ',
  ' Acronym ',
  ' Study Status ',
  ' Conditions ',
  ' Interventions ',
  ' Sponsor ',
  ' Collaborators ',
  ' Enrollment ',
  ' Funder Type ',
  ' Study Type ',
  ' Study Design,Start Date,Completion Date'],
 ['"NCT03630471 ',
  ' Effectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India ',
  ' PRIDE ',
  ' COMPLETED ',
  ' Mental Health Issue (E.G., Depression, Psychosis, Personality Disorder, Substance Abuse) ',
  " BEHAVIORAL: PRIDE 'Step 1' problem-solving intervention|BEHAVIORAL: Enhanced usual care ",
  ' Sangath ',
  ' Harvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine ',
  ' 250 ',
  ' OTHER ',
  ' INTERVENTIONAL ',
  ' Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR, OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT",20 August 2018,28 February 2019'],
 ['"NCT03400371 ',
  ' Biology of Juvenile Myoclonic Epilepsy ',
  ' BIO

In [0]:
df.show(1, truncate= False)

+-----------+-------------+---------+--------------+------------+---------------+---------+---------------+------------+-------------+------------+----------------------------------------+
|_1         |_2           |_3       |_4            |_5          |_6             |_7       |_8             |_9          |_10          |_11         |_12                                     |
+-----------+-------------+---------+--------------+------------+---------------+---------+---------------+------------+-------------+------------+----------------------------------------+
|NCT Number | Study Title | Acronym | Study Status | Conditions | Interventions | Sponsor | Collaborators | Enrollment | Funder Type | Study Type | Study Design,Start Date,Completion Date|
+-----------+-------------+---------+--------------+------------+---------------+---------+---------------+------------+-------------+------------+----------------------------------------+
only showing top 1 row

